In [1]:
import cv2
import torch

image = cv2.imread('image.jpg')
image = cv2.resize(image, (224,224))
image = torch.tensor(image, dtype=torch.float)
image = image.unsqueeze(0)
image = torch.cat((image, image),  dim = 0)
image.shape

torch.Size([2, 224, 224, 3])

In [2]:
from model import Layer

input_layer = Layer.Input_layer()
encoder_layer = Layer.Encoder_layer()
decoder_layer = Layer.Decoder_layer()
cra = Layer.CRA()
x = input_layer(image)

/home/quocviet/miniconda3/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
from transformers import ViTModel, DeiTForImageClassificationWithTeacher
import torch.nn as nn

model_trans = DeiTForImageClassificationWithTeacher.from_pretrained('facebook/deit-tiny-distilled-patch16-224')
list_layer = list(model_trans.children())
list_layer = list(list_layer[0].children())[1]
list_layer = list(list(list_layer.children())[0].children())
list_secquence = [nn.Sequential(i) for i in list_layer] # list 12 Encoder layers

In [14]:
model_trans

DeiTForImageClassificationWithTeacher(
  (deit): DeiTModel(
    (embeddings): DeiTEmbeddings(
      (patch_embeddings): DeiTPatchEmbeddings(
        (projection): Conv2d(3, 192, kernel_size=(16, 16), stride=(16, 16))
      )
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): DeiTEncoder(
      (layer): ModuleList(
        (0-11): 12 x DeiTLayer(
          (attention): DeiTAttention(
            (attention): DeiTSelfAttention(
              (query): Linear(in_features=192, out_features=192, bias=True)
              (key): Linear(in_features=192, out_features=192, bias=True)
              (value): Linear(in_features=192, out_features=192, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
            (output): DeiTSelfOutput(
              (dense): Linear(in_features=192, out_features=192, bias=True)
              (dropout): Dropout(p=0.0, inplace=False)
            )
          )
          (intermediate): DeiTIntermediate(
            (dense)

In [4]:
encoder_layer(x).shape

torch.Size([12, 2, 197, 192])

In [8]:
total_params = sum(p.numel() for p in encoder_layer.parameters())
total_params

0

In [6]:
x = input_layer(image)
print(x.shape)
x = encoder_layer(x)
# y = encoder_layer(x)
# x = x[:,1:,:]
# y = y[:,1:,:]
# # x = x.reshape(2, 14, 14, 768)
# # x = x.permute(0, 3, 1, 2)
# tmp = cra(x, y)
# tmp.shape

torch.Size([2, 197, 192])


In [7]:
tmp.shape

NameError: name 'tmp' is not defined

In [9]:
from model.ViT_CRA import ViT_CRA

model = ViT_CRA()
total_params = sum(p.numel() for p in model.parameters())

In [16]:
model(image)[1].shape

torch.Size([2, 1, 28, 28])